In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import gym
from collections import deque
import random
import time
import yaml
from klqr import *
from train_utils import *

In [2]:
with open('config.yml','r') as ymlfile:
    config = yaml.load(ymlfile)
    
tf.reset_default_graph()
sess = tf.InteractiveSession(config=tf.ConfigProto(log_device_placement=True))

In [3]:
env = gym.make('LQ-v0')
agent = klqr(config,sess)
agent.build_model()
train_agent(agent,env,max_timesteps=100000)

z shape: (?, 3)
********** Iteration 1 ************
	Last Episode Reward: -122768
********** Iteration 2 ************
	Last Episode Reward: -251250
********** Iteration 3 ************
	Last Episode Reward: -10421
********** Iteration 4 ************
	Last Episode Reward: -13132
********** Iteration 5 ************
[[ 4.7175884   2.5355196  -0.80097485]
 [ 2.5355196   2.5242977  -0.8128734 ]
 [-0.80097485 -0.8128734   1.1462948 ]]
	Last Episode Reward: -898
********** Iteration 6 ************
	Last Episode Reward: -1962427
********** Iteration 7 ************
	Last Episode Reward: -2066711
********** Iteration 8 ************
	Last Episode Reward: -2024696
********** Iteration 9 ************
	Last Episode Reward: -1553755
********** Iteration 10 ************
[[ 2.2230268   0.40118304 -1.3500311 ]
 [ 0.40118304  1.339439   -0.7751753 ]
 [-1.3500311  -0.7751753   1.9038198 ]]
	Last Episode Reward: -157800
********** Iteration 11 ************
	Last Episode Reward: -2232073
********** Iteration

[[ 2.1432269   0.06716754 -0.07827357]
 [ 0.06716754  2.2742634   0.3868028 ]
 [-0.07827357  0.3868028   1.6916862 ]]
	Last Episode Reward: -85
********** Iteration 96 ************
	Last Episode Reward: -137
********** Iteration 97 ************
	Last Episode Reward: -113
********** Iteration 98 ************
	Last Episode Reward: -112
********** Iteration 99 ************
	Last Episode Reward: -98
********** Iteration 100 ************
[[ 2.0996008   0.17108786 -0.02221721]
 [ 0.17108786  2.2097008   0.26909617]
 [-0.02221721  0.26909617  1.6343083 ]]
	Last Episode Reward: -120
********** Iteration 101 ************
	Last Episode Reward: -134
********** Iteration 102 ************
	Last Episode Reward: -113
********** Iteration 103 ************
	Last Episode Reward: -89
********** Iteration 104 ************
	Last Episode Reward: -109
********** Iteration 105 ************
[[2.0780826  0.26350945 0.03289714]
 [0.26350945 2.1333585  0.16832745]
 [0.03289714 0.16832745 1.6237141 ]]
	Last Episod

[[2.1598458  0.62604177 0.32634324]
 [0.62604177 1.9174247  0.28688878]
 [0.32634324 0.28688878 1.9203987 ]]
	Last Episode Reward: -133
********** Iteration 191 ************
	Last Episode Reward: -116
********** Iteration 192 ************
	Last Episode Reward: -178
********** Iteration 193 ************
	Last Episode Reward: -151
********** Iteration 194 ************
	Last Episode Reward: -174
********** Iteration 195 ************
[[2.1641765  0.6216904  0.28282577]
 [0.6216904  1.9295766  0.33078527]
 [0.28282577 0.33078527 1.9205961 ]]
	Last Episode Reward: -157
********** Iteration 196 ************
	Last Episode Reward: -158
********** Iteration 197 ************
	Last Episode Reward: -107
********** Iteration 198 ************
	Last Episode Reward: -148
********** Iteration 199 ************
	Last Episode Reward: -148
********** Iteration 200 ************
[[2.1738806  0.62239695 0.30924943]
 [0.62239695 1.9292613  0.28798974]
 [0.30924943 0.28798974 1.9566933 ]]
	Last Episode Reward: -

KeyboardInterrupt: 

In [ ]:
sess.run(agent.A)

In [ ]:
sess.run(agent.B)

In [4]:
sess.run(agent.Q)

array([[ 1.0189447 ,  0.08293143,  0.15776801],
       [ 0.08293143,  0.7517757 , -0.05224696],
       [ 0.15776801, -0.05224696,  0.9286399 ]], dtype=float32)

In [5]:
sess.run(agent.R)

array([[ 8.738761  , -0.03628063, -2.764276  ],
       [-0.03628063, 14.90458   , -0.02685213],
       [-2.764276  , -0.02685213,  8.3499365 ]], dtype=float32)